# Setup

## Mount Drive

Mount Google Drive. This will create a new folder, called `My Drive` under `/content/drive/`. It is a FUSE-like filesystem mounted over the network, so the average IO performance is quite slow.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Move into the `BiseNetv1` directory, so that we are in the correct directory for python imports

In [ ]:
%cd /content/drive/MyDrive/Universita/MLDL/BiseNetv1/

/content/drive/MyDrive/Universita/MLDL/BiseNetv1


In [ ]:
%pwd

'/content/drive/MyDrive/Universita/MLDL/BiseNetv1'

Use autoreload extension so that when we change an imported module, it will get autoreloaded.

In [ ]:
%load_ext autoreload
%autoreload 2

## Download the datasets

Download `CamVid` and `IDDA` datasets from a Google Cloud Storage bucket. Previously we downloaded them from Google Drive but there is (rightly so) a quite small download quota.

Files are downloaded to the local disk which has much faster IO than the mounted drive.

In [ ]:
%%time
%cd /content
%pwd
!gsutil cp gs://recsys-2021-bucket/CamVid.zip /content/CamVid.zip
!gsutil cp gs://recsys-2021-bucket/IDDA.zip /content/IDDA.zip
!unzip -q CamVid.zip
!unzip -q IDDA.zip
!cp /content/drive/MyDrive/Universita/MLDL/classes_info.json /content/IDDA/classes_info.json
%cd /content/drive/MyDrive/Universita/MLDL/BiseNetv1/

/content
Copying gs://recsys-2021-bucket/CamVid.zip...
- [1 files][579.3 MiB/579.3 MiB]                                                
Operation completed over 1 objects/579.3 MiB.                                    
Copying gs://recsys-2021-bucket/IDDA.zip...
- [1 files][  4.9 GiB/  4.9 GiB]   50.3 MiB/s                                   
Operation completed over 1 objects/4.9 GiB.                                      
/content/drive/MyDrive/Universita/MLDL/BiseNetv1
CPU times: user 1.37 s, sys: 208 ms, total: 1.58 s
Wall time: 2min 44s


# Launch the training script

In [ ]:
!python train_FDA_05.py

begin epoch 1
100% 234/234 [03:15<00:00,  1.20it/s]
begin epoch 2
100% 234/234 [03:12<00:00,  1.21it/s]
begin epoch 3
100% 234/234 [03:10<00:00,  1.23it/s]
begin epoch 4
100% 234/234 [03:07<00:00,  1.25it/s]
begin epoch 5
100% 234/234 [03:08<00:00,  1.24it/s]
begin epoch 6
100% 234/234 [03:11<00:00,  1.22it/s]
begin epoch 7
100% 234/234 [03:11<00:00,  1.22it/s]
begin epoch 8
100% 234/234 [03:11<00:00,  1.22it/s]
begin epoch 9
100% 234/234 [03:09<00:00,  1.24it/s]
begin epoch 10
100% 234/234 [03:08<00:00,  1.24it/s]
begin epoch 11
100% 234/234 [03:09<00:00,  1.23it/s]
begin epoch 12
100% 234/234 [03:09<00:00,  1.23it/s]
begin epoch 13
100% 234/234 [03:09<00:00,  1.24it/s]
begin epoch 14
100% 234/234 [03:08<00:00,  1.24it/s]
begin epoch 15
100% 234/234 [03:09<00:00,  1.24it/s]
begin epoch 16
100% 234/234 [03:10<00:00,  1.23it/s]
begin epoch 17
100% 234/234 [03:10<00:00,  1.23it/s]
begin epoch 18
100% 234/234 [03:12<00:00,  1.21it/s]
begin epoch 19
100% 234/234 [03:10<00:00,  1.23it/s]
be

# Validation

In [ ]:
import os
import gc

from IPython.core.display import display, HTML

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from model.build_BiSeNet import BiSeNet
from dataset.IDDA import IDDA
from dataset.CamVid import CamVid
NUM_CLASSES = 12

CROP_HEIGHT = 720
CROP_WIDTH = 960

CONTEXT_PATH = 'resnet101'

LEARNING_RATE_SEGMENTATION = 2.5e-4
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9

LEARNING_RATE_DISCRIMINATOR = 1e-4

POWER = 0.9

NUM_STEPS = 250000
ITER_SIZE = 1

CHECKPOINT_STEP = 1
CHECKPOINT_PATH = './checkpoint_101_adversarial_both_augmentation_epoch_len_IDDA/'

BATCH_SIZE_CAMVID = 2
BATCH_SIZE_IDDA = 2

CAMVID_PATH = ['/content/CamVid/train/', '/content/CamVid/val/']
CAMVID_TEST_PATH = ['/content/CamVid/test/']
CAMVID_LABEL_PATH = ['/content/CamVid/train_labels/', '/content/CamVid/val_labels/']
CAMVID_TEST_LABEL_PATH = ['/content/CamVid/test_labels/']
CSV_CAMVID_PATH = '/content/CamVid/class_dict.csv'

IDDA_PATH = '/content/IDDA/rgb/'
IDDA_LABEL_PATH = '/content/IDDA/labels'
JSON_IDDA_PATH = '/content/IDDA/classes_info.json'

NUM_WORKERS = 0

LOSS = 'dice'

SOURCE_LABEL = 0
TARGET_LABEL = 1
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()
%config InlineBackend.figure_format = 'retina'
from utils import reverse_one_hot, colour_code_segmentation
from torchvision import transforms as T

mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
unnormalize = T.Normalize((-mean/std).tolist(), (1.0 / std).tolist())

In [ ]:
!pip3 install tensorboardX

     |████████████████████████████████| 122kB 13.2MB/s 


In [ ]:
from dataclasses import dataclass
@dataclass
class MyArgs:
  num_classes: int
  use_gpu: bool
  loss: str
from train import val

In [ ]:
!nvidia-smi

Thu Jun 17 21:49:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = BiSeNet(NUM_CLASSES, CONTEXT_PATH).cuda()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Universita/MLDL/BiseNetv1/checkpointBeta05/50_0.05_fda.pth'))

<All keys matched successfully>

In [ ]:
model.eval()
None

In [ ]:
%%time
target_dataset_test = CamVid(
  image_path=CAMVID_TEST_PATH,
  label_path= CAMVID_TEST_LABEL_PATH,csv_path= CSV_CAMVID_PATH,
  scale=(CROP_HEIGHT, CROP_WIDTH),
  loss=LOSS,
  mode='val'
)
target_dataloader_test = DataLoader(
  target_dataset_test,
  batch_size=1,
  shuffle=True,
  num_workers=NUM_WORKERS,
  drop_last=True,
  pin_memory=True
)
target_dataloader_test_iter = iter(target_dataloader_test)
val(MyArgs(NUM_CLASSES, True, 'dice'), model, target_dataloader_test_iter)

start val!
precision per pixel for test: 0.703
mIoU for validation: 0.327
CPU times: user 4min 14s, sys: 973 ms, total: 4min 15s
Wall time: 4min
